# Do it 자연어 처리 3장 숫자 세계로 떠난 자연어
## 3.4 트랜스포머에 적용된 기술들
### 피드포워드 뉴럴 네트워크

In [1]:
# 피드포워드 뉴럴 네트워크 계산 예시
import torch
x = torch.tensor([2, 1])
w1 = torch.tensor([[3,2,-4], [2,-3,1]])
b1 = 1
w2 = torch.tensor([[-1, 1], [1,2], [3,1]])
b2 = -1

In [2]:
h_preact = torch.matmul(x, w1) + b1 # 행렬곱 + 바이어스 수행
h = torch.nn.functional.relu(h_preact) # 활성함수 거침
y = torch.matmul(h, w2) + b2 # 두번째 은닉층 수행

In [3]:
print(h_preact)
print(h)
print(y)

tensor([ 9,  2, -6])
tensor([9, 2, 0])
tensor([-8, 12])


### 잔차연결
- 모델이 다양한 관점에서 블록 계산을 수행
- 모델 중간에 블록을 건너뛰는 경로를 설정함으로써 학습을 쉽게 하는 효과를 거둘 수 있음

### 레이어 정규화
- 학습이 안정되고 그 속도가 빨라짐

In [5]:
# 레이어 정규화 예시
import torch
input = torch.tensor([[1.0, 2.0, 3.0], [1.0, 1.0, 1.0]])
m = torch.nn.LayerNorm(input.shape[-1]) # 레이어 정규화 코드
output = m(input)
print(output)
print(m.weight)
print(m.bias)

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)
Parameter containing:
tensor([1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


### 드롭아웃
- 과적합 방지
- 일부 노드를 확률적으로 0으로 대치하여 계산에서 제외
- 10%가 일반적

In [6]:
# 드롭 아웃 예시
import torch
m = torch.nn.Dropout(p=0.2) # 안정적인 학습을 위해 각 요솟값에 1/(1-p)를 곱하기도 함
input = torch.randn(1,10)
output = m(input)
print(input)
print(output)

tensor([[ 0.3482,  1.7226,  0.9384,  0.2712,  2.2820,  0.9701,  0.3257,  1.5054,
         -1.4174, -0.7939]])
tensor([[ 0.4352,  2.1533,  1.1730,  0.0000,  2.8525,  0.0000,  0.4071,  1.8818,
         -1.7717, -0.9924]])


### 아담 옵티마이저
- 딥러닝 모델 학습: 출력-정답 사이 오차 최소화, 파라미터 업데이트 과정
- 손실(loss): 오차
- 기울기(gradient): 오차를 최소화하는 방향
- 최적화(optimization): 오차를 최소화하는 과정

In [ ]:
# 아담 옵티마이저 예시
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=model.learning_rate) 
# lr을 정해주면 아담이 model.parameters(최적화 대상 파라미터들)에 방향과 보폭을 정해줌

---
## 3.5 Bert와 GPT 비교
### GPT
- 언어모델
- 일방향성
- 문장 생성에 적합
- 디코더만 취해 사용
### Bert
- 마스크 언어 모델
- 양방향성
- 문장의 의미 추출하는데 적합
- 인코더만 취해 사용


---
## 3.6 단어/문장을 벡터로 변환
### 파인튜닝
- 프리트레인 모델과 그 위 작은 모듈을 포함한 전체 모델을 다운스트림 데이터로 업데이트 하는 과정

### 문장 벡터 활용 : 문서 분류
- pooler_out 벡터(최종출력)를 만들어 하나로 응집 시킴
- 긍정, 부정, 중립 등 확률을 배정하고 정답과 가까워지게 함 -> 파인튜닝

### 단어 벡터 활용 : 개체명 인식
- 마지막 블록이 모든 단어 벡터를 활용
- 각 개체명 범주가 될 확률을 만들고 정답과 가까워지게 함

### 문장을 벡터로 변환하기

In [8]:
# 토크나이저 선언
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

vocab.txt: 100%|██████████| 250k/250k [00:00<00:00, 2.18MB/s]
tokenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<00:00, 48.6kB/s]
config.json: 100%|██████████| 619/619 [00:00<00:00, 1.69MB/s]


In [9]:
# 모델 선언
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config = pretrained_model_config,
)

pytorch_model.bin: 100%|██████████| 438M/438M [00:07<00:00, 56.2MB/s] 
Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [12]:
# 입력값 만들기
sentences = ['안녕하세요', '하이!']
features = tokenizer(
    sentences,
    max_length=10,
    padding='max_length',
    truncation=True,
)

In [13]:
features
# 2, 3은 스페셜 토큰 CLS, SEP

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [14]:
# 피처를 토치 텐서로 변환
features = {k: torch.tensor(v) for k, v in features.items()}

In [15]:
# 임베딩 계산하기
outputs = model(**features)

In [16]:
# Bert 모델의 여러 출력 결과를 한데 묶은 것
# 안녕하세요, 하이!의 입력 토큰 각각에 해당하는 Bert의 마지막 레이어 출력 벡터들
outputs.last_hidden_state

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

In [17]:
# 문장 2개에 속한 각각의 토큰(길이 10)이 768차원짜리의 벡터로 변환
outputs.last_hidden_state.shape

torch.Size([2, 10, 768])

In [18]:
# 문장 수준 벡터
outputs.pooler_output

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

In [19]:
# 문장 2개에 속한 각각의 토큰이 768차원짜리의 벡터로 변환
outputs.pooler_output.shape

torch.Size([2, 768])

- 단어 수준 벡터 임베딩: outputs.last_hidden_state
- 문장 수준 벡터 임베딩: outputs.pooler_output

### 태스크 모듈 만들기
- 파인 튜닝을 수행하기 위해 태스크 수행을 위한 작은 모듈을 추가해야함
- 다운 스트림 태스크 별로 조금씩 달라짐